<a href="https://colab.research.google.com/github/yuliiabosher/Cyber_Resilience_Course/blob/main/CDProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Installing the googlemaps library

In [ ]:
!pip install googlemaps

  Preparing metadata (setup.py) ... done
  Created wheel for googlemaps: filename=googlemaps-4.10.0-py3-none-any.whl size=40712 sha256=87b9bff3439c1d8807f0add34ef9a0b47b37ad8bd31d3358c7b560b78ded4601
  Stored in directory: /root/.cache/pip/wheels/17/f8/79/999d5d37118fd35d7219ef57933eb9d09886c4c4503a800f84
Successfully built googlemaps


#### Importing the required modules

In [ ]:
import googlemaps
import os
import requests
import json
from google.colab import output

#### Setting an environment variable for the Google API key

In [ ]:
google_key = input("Enter the Google maps API key: ")
os.environ['GOOGLE_API_KEY'] = google_key
output.clear()

#### Defining the get_schools(lat,lon,rad) function
---
The function returns a lists of lists. Each list contains a school name and its coordinates. The schools returned are all within the radius from the coordinates passed to the function

In [ ]:
gmaps = googlemaps.Client(os.environ.get('GOOGLE_API_KEY'))

def get_schools(lat,lon,rad):
  school_list=[]
  try:
    places_result = gmaps.places_nearby(location= f"{lat},{lon}", radius = rad, type = "school")
    for place in places_result['results']:
      school_name= place['name']
      school_location_lat= place['geometry']['location']['lat']
      school_location_lng= place['geometry']['location']['lng']
      school_list.append([school_name, school_location_lat,school_location_lng])
    return school_list
  except googlemaps.exceptions.ApiError as e:
    if e.status=="INVALID_REQUEST":
      print("coordinates are invalid")
  except Exception as e:
    print(f"Error: {e}")

#### Setting an environment variable for the AWS API key

In [ ]:
aws_key = input("Enter the AWS API key: ")
os.environ['AWS_API_KEY'] = aws_key
output.clear()

#### Defining an add_schools(lat,lng,rad) function
---
The function add_schools(lat,lng,rad) calls a Lambda function's API gateway using POST method. It returns a string that contains a message and a list of lists with newly added school names and coordinates

In [ ]:
def add_schools(lat,lng,rad):
  url = "https://a595c6ya39.execute-api.eu-west-2.amazonaws.com/default/myPythonProjectFunc"

  school_data = get_schools(lat,lng,rad)

  payload = json.dumps({
    "data": school_data
  })
  headers = {
    'Content-Type': 'application/json',
    'x-api-key': os.environ.get('AWS_API_KEY')
  }

  response = requests.request("POST", url, headers=headers, data=payload)
  return response.text

#### Defining a show_schools() function
---
The function show_schools() calls the Lambda function's API gateway using GET method. It returns a string that contains a message and a list of lists with school names and coordinates stored in the S3 bucket file

In [ ]:
def show_schools():
  url = "https://a595c6ya39.execute-api.eu-west-2.amazonaws.com/default/myPythonProjectFunc"
  payload = ""
  headers = {
    'x-api-key': os.environ.get('AWS_API_KEY')
  }

  response = requests.request("GET", url, headers=headers, data=payload)
  return response.text

#### Defining a select_an_action() function
---
The user can interact with the application through the function select_an_action(). It offers the user two options. The first option is to add new schools to the file. If selected it prompts the user to input the coordinates of a location and a radius. It will then return a list of lists with newly added schools that were not in the file before. The second option is to retrieve the schools and their coordinates currently stored in an S3 bucket file. The data returned is the list of lists where each school name and its coordinates are stored in a separate list.

In [56]:
def select_an_action():
  try:
      action = input('select an option \n 1 for add new schools \n 2 for show all schools ')
      if action == "1":
        latitude = float(input('enter a latitude as a decimal '))
        longitude = float(input('enter a longutude as a decimal '))
        radius = float(input('enter a radius as an integer '))
        newly_added_schools = json.loads(add_schools(latitude,longitude,radius))
        return newly_added_schools['data']
      elif action == "2":
        all_schools = json.loads(show_schools())
        return all_schools['data']
      else:
        return 'you should have entered 1 or 2'
  except Exception as e:
    return f"There was an error {e}"
prompt = select_an_action()
print(prompt)

select an option 
 1 for add new schools 
 2 for show all schools 1
enter a latitude as a decimal 55.858700
enter a longutude as a decimal -4.295905
enter a radius as an integer 4000
[["Hutchesons' Grammar School", 55.83644289999999, -4.282187], ["St Aloysius' College", 55.8668397, -4.2633366], ["St Saviour's Primary School (Govan)", 55.86065359999999, -4.3063033], ['Kelvin School', 55.8668041, -4.2941658], ['Glasgow Club Bellahouston', 55.8459204, -4.3239934], ['Royal Conservatoire of Scotland', 55.8661893, -4.257588699999999], ['Kinning Park Nursery School', 55.85025349999999, -4.283776200000001], ['Adelaides Nursery', 55.8646524, -4.264747], ['Nuffield Health Glasgow Minerva Fitness & Wellbeing Gym', 55.8628797, -4.2854098], ['Nuffield Health Glasgow Central Fitness & Wellbeing Gym', 55.85920369999999, -4.2811215], ['Gisoel Glasgow', 55.86045559999999, -4.259549000000001], ['Kelvinside Academy', 55.8829681, -4.2950843], ['Bellahouston Academy', 55.8481055, -4.3005388], ['CC MUSIC', 